
* `LangSmith` is a platform for building production-grade LLM applications.
* It lets you debug, test, evaluate, and monitor chains and intelligent agents built on any LLM framework and seamlessly integrates with LangChain, the go-to open source framework for building with LLMs.

* LangSmith provides several benefits for developing and evaluating conversational AI systems:

  - Accelerates debugging of new conversation flows, agents, and toolsets.
  -Enables visualization of relationships between components like chains, models, and retrievers.
  - Supports testing different prompts and models for a specific module.
  - Allows running chains repeatedly over datasets to assess consistency.
  - Captures usage data to generate insights using analytics.

# 10.1 Getting Started

In [1]:
!pip3 install -U langsmith

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 122.4 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.33
    Uninstalling langsmith-0.1.33:
      Successfully uninstalled langsmith-0.1.33

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


**How to Get the required KEYS**

* LangSmith API Key : https://docs.smith.langchain.com/
* LangChain API Key : https://smith.langchain.com/settings

* Serper API Key : https://serper.dev/

In [4]:
import os
from uuid import uuid4

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Getting Started"
os.environ["OPENAI_API_KEY"] = "Enter your API Key"
os.environ["SERPER_API_KEY"] = "Enter your API Key"

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "ls__446246ca5f6744558aedc927aa852c05"  # Update with your API key



In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?')

Check the LangSmith Platform for live tracking of your Actions. Click here https://smith.langchain.com/

### 10.2 OpenAI Q&A

In [6]:
chat_model = ChatOpenAI() # temperature=0.0
chat_model

print(chat_model.predict("Hi !! Who is Dr.APJ Abdul Kalam ?"))

Dr. APJ Abdul Kalam, full name Avul Pakir Jainulabdeen Abdul Kalam, was an Indian scientist, politician, and the 11th President of India. He was born on October 15, 1931, in Rameswaram, Tamil Nadu, and died on July 27, 2015, in Shillong, Meghalaya. 

Dr. Kalam was known as the "Missile Man of India" for his significant contributions to India's space and missile programs. He played a crucial role in the development of India's first indigenous satellite launch vehicle (SLV-III), which successfully deployed the Rohini satellite in near-earth orbit in 1980. He also played a key role in the development of the Agni and Prithvi missiles.

Apart from his scientific achievements, Dr. Kalam served as the President of India from 2002 to 2007. During his presidency, he focused on promoting education, especially in rural areas, and inspiring the youth of India to pursue scientific research and innovation.

Dr. APJ Abdul Kalam was widely respected and admired for his humility, simplicity, and dedica

### 10.3 Integrate LangSmith with LangChain Chat Bot

In [8]:
import openai
import os
import sys
sys.path.append('../..')

In [9]:
from langchain.document_loaders import PyPDFLoader

# Load PDF
loader = PyPDFLoader("/content/META-Q2-2023-Earnings-Call-Transcript.pdf")
docs = loader.load()

In [10]:
docs[0]

Document(page_content="1 \n Meta Platforms , Inc. ( META ) \nSecond Quarter 2023 Results Conference Call  \nJuly 26th, 202 3 \n \nKen Dorell , Director , Investor Relations  \n \nThank you. Good afternoon and welcome to Meta Platforms second quarter 2023 earnings conference \ncall. Joining me today to discuss our results are Mark Zuckerberg, CEO and Susan Li, CFO.  \n \nBefore we get started, I would like to take this opportunity to remind you that our remarks today will \ninclude forward‐looking statements. Actual results may differ materially from t hose contemplated by \nthese forward‐looking statements.  \n \nFactors that could cause these results to differ materially are set forth in today’s press release, and in \nour quarterly report on form 10 -Q filed with the SEC. Any forward‐looking statements that we make on \nthis call are based on assumptions as of today and we undertake no obligation to update these \nstatements as a result of new information or future events.  \n \nDuri

In [11]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [12]:
splits = text_splitter.split_documents(docs)
len(splits)

55

In [13]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'docs/chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

55


In [14]:
from langchain.chat_models import ChatOpenAI

llm_name = 'gpt-3.5-turbo'
llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [17]:
# Build prompt

from langchain.prompts import PromptTemplate
template = """ Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer: """

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)

In [18]:
# Run chain
from langchain.chains import RetrievalQA
question = "what is the total revenue ? "
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})


result = qa_chain({"query": question})
result["result"]

'The total revenue for Q2 was $32.0 billion. Thanks for asking!'

### 10.4 Agents - Search

In [37]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=12653af3d7cfb6c4b41afbe1f75496163631d4388fe809a6fac254f94988690b
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [39]:
from langchain.tools import DuckDuckGoSearchRun
from langchain.agents import Tool
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [40]:
QUERY = "Who is the Hitman of Indian Cricket Team ? What is his age multiplied by 2 ?"

In [47]:
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003" ,temperature=0)
tools = load_tools(["google-serper", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [48]:
agent.run(QUERY)



> Entering new AgentExecutor chain...
 I need to find out who the Hitman is and then calculate his age multiplied by 2.
Action: google_serper
Action Input: "Hitman of Indian Cricket Team"
Observation: This achievement further solidifies his status as one of India's premier batsmen and a cricketing legend. On the 22nd of October 2023, during a thrilling encounter against New Zealand, Rohit Sharma, fondly known as the 'Hitman,' etched his name in the annals of cricket history.
Thought: I need to find out Rohit Sharma's age.
Action: google_serper
Action Input: "Rohit Sharma age"
Observation: 36 years
Thought: I now know the final answer
Final Answer: 72

> Finished chain.


'72'